In [1]:
import tweepy
import numpy as np
import pandas as pd

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from config import consumer_key,consumer_secret, access_token, access_token_secret

In [2]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

# Setup Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Target Search Term
target_terms = [
    "@SouthwestAir",
    "@AmericanAir",
    "@SpiritAirlines",
    "@Virginatlantic",
    "@Delta",
    "@FlyAirNZ",
]

# "Real Person" Filters
min_tweets = 5
max_tweets = 10000
max_followers = 2500
max_following = 2500
lang = "en"

# List to hold results
results_list = []

# Loop through all target users
for target in target_terms:

    # Variable for holding the oldest tweet
    oldest_tweet = None

    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    # Loop through 10 times
    for x in range(10):

        # Run search around each tweet
        tweets = api.search(target, count=100, result_type="recent", max_id=oldest_tweet)

        # Loop through all tweets
        for tweet in tweets["statuses"]:

            # Use filters to check if user meets conditions
            if (tweet["user"]["followers_count"] < max_followers
                and tweet["user"]["statuses_count"] > min_tweets
                and tweet["user"]["statuses_count"] < max_tweets
                and tweet["user"]["friends_count"] < max_following
                and tweet["user"]["lang"] == lang):

                # Run Vader Analysis on each tweet
                results = analyzer.polarity_scores(tweet["text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]

                # Add each value to the appropriate list
                compound_list.append(compound)
                positive_list.append(pos)
                negative_list.append(neg)
                neutral_list.append(neu)
                
            # Set the new oldest_tweet value
            oldest_tweet = tweet["id"] - 1

    # Store the Average Sentiments
    sentiment = {
        "User": target,
        "Compound": np.mean(compound_list),
        "Positive": np.mean(positive_list),
        "Neutral": np.mean(negative_list),
        "Negative": np.mean(neutral_list),
        "Tweet Count": len(compound_list)
    }

    # Print the Sentiments
    print(sentiment)
    print()
    
    # Append airline results to 'results_list'
    results_list.append(sentiment)

{'User': '@SouthwestAir', 'Compound': 0.15849308681672025, 'Positive': 0.11766237942122187, 'Neutral': 0.046152733118971064, 'Negative': 0.8361913183279742, 'Tweet Count': 622}

{'User': '@AmericanAir', 'Compound': -0.00843502824858757, 'Positive': 0.08882674199623353, 'Neutral': 0.09124105461393596, 'Negative': 0.8199322033898305, 'Tweet Count': 531}

{'User': '@SpiritAirlines', 'Compound': -0.009684323432343236, 'Positive': 0.09629042904290429, 'Neutral': 0.09994389438943894, 'Negative': 0.8037590759075908, 'Tweet Count': 606}

{'User': '@Virginatlantic', 'Compound': 0.20328805147058823, 'Positive': 0.1391525735294118, 'Neutral': 0.039213235294117646, 'Negative': 0.8216341911764706, 'Tweet Count': 544}

{'User': '@Delta', 'Compound': 0.06507427597955706, 'Positive': 0.09397103918228279, 'Neutral': 0.06412606473594548, 'Negative': 0.8418960817717207, 'Tweet Count': 587}

{'User': '@FlyAirNZ', 'Compound': 0.2911721649484536, 'Positive': 0.17019381443298973, 'Neutral': 0.034705154639175

In [4]:
airline_df = pd.DataFrame(results_list).set_index("User").round(3)
airline_df

,Compound,Negative,Neutral,Positive,Tweet Count
User,,,,,
@SouthwestAir,0.158,0.836,0.046,0.118,622
@AmericanAir,-0.008,0.820,0.091,0.089,531
@SpiritAirlines,-0.010,0.804,0.100,0.096,606
@Virginatlantic,0.203,0.822,0.039,0.139,544
@Delta,0.065,0.842,0.064,0.094,587
@FlyAirNZ,0.291,0.795,0.035,0.170,485
